<a href="https://colab.research.google.com/github/RudkovYaroslav/Selenium_Riot/blob/main/Selenium_Riot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Проект - "Сбор данных для челленджа League of Legends"

Цель проекта: Существует достижение в игре LOL - иметь более 10000 очков мастерства на каждом из 150 чемпионов. Для того, чтобы отслеживать прогресс, необходимо создать excel файл, который будет показывать героев, их текущие очки мастерства, а также позиции на которых они играются. Для сбора данных будем использовать популярный сайт статистики по LOL - [op.gg](https://www.op.gg/)

Ход исследования:

 - Создадим первоначальные cookie для того, чтобы пропускать дальнейшую регистрацию каждый раз, когда хотим собрать информацию о мастерстве
 - Возьмем данные он текущем мастерстве всех чемпионов со страницы пользователя
 - Возьмем данные о позициях чемпионов, используя главную страницу
 - Объединим две таблица по именам чемпионов
 - Разъединим таблицу по позициям для того, чтобы отобразить их на отдельных листах книги excel
 - Запишем все в файл

## Импорт библиотек и предисловие

In [ ]:
from selenium import webdriver #Основная библиотека для получения данных
from selenium.webdriver.common.by import By #Часть Selenium
import time #Библиотека для постановки таймера
import pandas as pd #Основная библиотека для создания DataFrame
import regex as re #Библиотека для более углубленного поиска
from selenium.webdriver.chrome.service import Service #Часть Selenium
from selenium.webdriver.chrome.options import Options #Часть Selenium
import pickle #Используется для cookie
from bs4 import BeautifulSoup #Библиотека для углубленного поиска в html коде
import requests #Библиотека для написания get запросов(получения html кода из ссылки)

Основные проблемы, с которыми пришлось столкнуться:

 - Ни BeatifulSoup, ни requests.get не могут получить правильные данные со страниц сайта [op.gg](https://www.op.gg/) из-за того, что сайт, после того, как на него заходит пользователь, сам отправляет get запрос на API игры(который доступен только для специальных организаций). Соответственно, используется библиотека Selenium.
 - Просмотр информации о пользователе требует авторизации. Чтобы каждый раз не повторять этот процесс, единоразово создали cookie и записали их в отдельный файл.
 - Очень важным шагом является определение позиции, на которой играет каждый чемпион. Получение этой информации оказалось трудным. Она расположена только на странице с конкретным чемпионом. Поэтому пришлось взять все ссылки на страницы чемпионов и на каждой взять позицию

## Создание функций

Создадим функции, которые будем использовать для запросов

In [ ]:
def cook():
    # Путь к драйверу браузера
    cService = webdriver.ChromeService('C:/Users/User/Downloads/yandexdriver-24.7.0.2299-win64/yandexdriver.exe')

    # Ссылка, с которой будем брать данные
    url = "https://www.op.gg/summoners/euw/SatoruGоjo-2031/mastery"

    # Запуск браузера
    driver = webdriver.Chrome(service = cService)
    driver.get(url)

    # Даем пользователю время для ручной авторизации
    input("Пожалуйста, авторизуйтесь на сайте вручную, затем нажмите Enter для продолжения...")

    # Сохраняем cookies в файл
    cookies = driver.get_cookies()
    with open("cookies.pkl", "wb") as file:
        pickle.dump(cookies, file)

    print("Cookies успешно сохранены в файл!")

    # Закрываем браузер
    driver.quit()
    return 0

Функция открывает сайт и ждет, пока мы залогинимся, затем сохраняет куки

In [ ]:
def selen():
    mastery, champ = [], []
    # Путь к драйверу браузера
    cService = webdriver.ChromeService('C:/Users/User/Downloads/yandexdriver-24.7.0.2299-win64/yandexdriver.exe')

    # Ссылка, с которой будем брать данные
    url = "https://www.op.gg/summoners/euw/SatoruGоjo-2031/mastery"

    # Запуск браузера
    driver = webdriver.Chrome(service = cService)
    driver.get(url)

    # Путь к файлу с cookie
    cookies_file = "cookies.pkl"

    # Загружаем файл cookie
    with open("cookies.pkl", "rb") as file:
        cookies = pickle.load(file)
        for cookie in cookies:
            driver.add_cookie(cookie)

    # Загружаем нужную страницу после добавления cookie
    driver.get('https://www.op.gg/summoners/euw/SatoruGоjo-2031/mastery')

    # Ожидание для полной загрузки страницы.
    time.sleep(5)

    #Основной запрос
    mastery_points_list = driver.find_elements(By.CLASS_NAME, "champion-point") #Берем очки мастерства
    for elem in mastery_points_list:
        mastery.append(elem.text)
    champ_name_list = driver.find_elements(By.CLASS_NAME, "champion-name") #Берем имена чемпионов
    for elemen in champ_name_list:
        champ.append(elemen.text)
    driver.quit()
    return(mastery, champ)

Основной запрос. Используя Selenium, открываем страницу пользователя и считываем всю информацию о мастерстве и имени чемпиона

In [ ]:
def champ():
    # Путь к драйверу браузера
    cService = webdriver.ChromeService('C:/Users/User/Downloads/yandexdriver-24.7.0.2299-win64/yandexdriver.exe')

    # Ссылка, с которой будем брать данные
    url = "https://www.op.gg/champions"

    # Запуск браузера
    driver = webdriver.Chrome(service = cService)
    driver.get(url)

    # Ставим ожидание
    driver.implicitly_wait(5)

    # Берем нужные нам классы
    elements = driver.find_elements(By.CSS_SELECTOR, '.css-mtyeel.e191lqlp2')

# Сбор всех связанных ссылок
    name, position = [], []
    for element in elements:
        name.append(element.text) # Добавляем имя персонажа
        parent_link = element.find_element(By.XPATH, './ancestor::a') # Ссылка на чемпиона находится выше, чем класс, который мы получили
        link = parent_link.get_attribute('href')
        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'html.parser') # Получили html код страницы с чемпионом
        gp = soup.find_all("div", {'data-key' : 'FILTER-POSITION'}) # Ищем все позиции
        data_value = []
        for item in gp:
            data_value.append(item['data-value'])
        position.append(data_value)
    driver.quit()
    return(name, position)

Второй запрос. Открываем главную страницу, для каждого чемпиона берем ссылку на страницу с ним, оттуда берем все позиции, на коорых он играет

## Использование запросов

In [ ]:
#cook()

Пожалуйста, авторизуйтесь на сайте вручную, затем нажмите Enter для продолжения... 


Cookies успешно сохранены в файл!


0

Единоразово используем функцию для создания файла cookie

In [ ]:
mastery, champ = selen()

In [ ]:
df = pd.DataFrame({'champ' : champ, 'mastery': mastery})

In [ ]:
name, position = champ()

In [ ]:
pos_df = pd.DataFrame({'champ' : name, 'position': position})

In [ ]:
merged_df = pd.merge(df, pos_df, on = 'champ')

Получили объединенный DataFrame, проверим, правильно ли все отображается

In [ ]:
merged_df

,champ,mastery,position
0,Джин,607856,[adc]
1,Акшан,160739,[mid]
2,Орн,93994,[top]
3,Экко,93036,"[jungle, mid]"
4,Наутилус,90880,[support]
...,...,...,...
163,Таам Кенч,965,"[top, support]"
164,Физз,965,[mid]
165,Триндамир,925,[top]
166,К'Санте,889,[top]


Все как надо

## Обработка полученного DataFrame

Для сортировки нам надо превратить цифровые значения в int

In [ ]:
merged_df['mastery'] = merged_df['mastery'].str.replace(' ', '')

In [ ]:
merged_df['mastery'] = merged_df['mastery'].astype(int)

Создадим black_list - это мой личный лист из тех 18 чемпионов, которых можно пропустить(для достижения необходимо лишь 150, когда на данный момент доступно 168)

In [ ]:
black_list = ['Афелий', 'Бард', 'Дрейвен', 'Элиза', 'Иверн', 'Каллиста', 'Клед', 'Нидали', 'Киана', 'Рената Гласк', 'Ренектон',
             'Ренгар', 'Ривен', 'Шако', 'Синджед', 'Зои']

## Запись в excel файл

Создадим 5 листов, которые разграничены по позициям чемпиона, также отсечем всех чемпионов, которые уже "пройдены"

In [ ]:
df_top = merged_df[(df['mastery'] < 10000) & (~merged_df['champ'].isin(black_list))&(merged_df.position.apply(lambda x: 'top' in x))]

In [ ]:
df_sup = merged_df[(merged_df['mastery'] < 10000) & (~merged_df['champ'].isin(black_list))&(merged_df.position.apply(lambda x: 'support' in x))]

In [ ]:
df_adc = merged_df[(merged_df['mastery'] < 10000) & (~merged_df['champ'].isin(black_list))&(merged_df.position.apply(lambda x: 'adc' in x))]

In [ ]:
df_mid = merged_df[(merged_df['mastery'] < 10000) & (~merged_df['champ'].isin(black_list))&(merged_df.position.apply(lambda x: 'mid' in x))]

In [ ]:
df_jungle = merged_df[(merged_df['mastery'] < 10000) & (~merged_df['champ'].isin(black_list))&(merged_df.position.apply(lambda x: 'jungle' in x))]

И запишем все в файл

In [ ]:
with pd.ExcelWriter(r'C:\Users\User\Desktop\league.xlsx') as writer:
    df_top.to_excel(writer, sheet_name='Top')
    df_sup.to_excel(writer, sheet_name='Sup')
    df_jungle.to_excel(writer, sheet_name='Jungle')
    df_mid.to_excel(writer, sheet_name='Mid')
    df_adc.to_excel(writer, sheet_name='Adc')

Вывод:

 - Создали excel файл, в котором находится вся нужная информация для более удобного получения достижения: Имя чемпиона, количество очков мастерства, а также разграничение по позициям, на которых чемпион играет